In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date, datetime
import holidays

In [2]:
lanes = pd.read_csv('tmcs_2020_2029_lanes.csv', index_col = 0)

data = pd.read_csv('tmcs_2020_2029.csv')

data_clean = pd.read_csv('tmcs_2020_2029_clean.csv')
graph = pd.read_csv('graph.csv')

In [3]:
lanes_col = []
oneway_col = []
for id in data.location_id:
    lanes_col.append(lanes.loc[id].lanes)
    if (np.isnan(lanes.loc[id].one_way)):
        oneway_col.append(False)
    else:
        oneway_col.append(True)

In [3]:
data_clean.head()

,location_id,year,month,day,time_start_hour,time_start_min,time_end_hour,time_end_min,num_lanes,is_oneway,...,nb_l,sb_r,sb_t,sb_l,eb_r,eb_t,eb_l,wb_r,wb_t,wb_l
0,13060,2020,1,8,7.0,30.0,7.0,45.0,2,False,...,0.0,0.0,125.0,15.0,0.0,0.0,0.0,6.0,0.0,6.0
1,13060,2020,1,8,7.0,45.0,8.0,0.0,2,False,...,0.0,0.0,117.0,10.0,0.0,0.0,0.0,10.0,0.0,4.0
2,13060,2020,1,8,8.0,0.0,8.0,15.0,2,False,...,0.0,0.0,134.0,17.0,0.0,0.0,0.0,13.0,0.0,10.0
3,13060,2020,1,8,8.0,15.0,8.0,30.0,2,False,...,0.0,0.0,161.0,9.0,0.0,0.0,0.0,15.0,0.0,9.0
4,13060,2020,1,8,8.0,30.0,8.0,45.0,2,False,...,0.0,0.0,139.0,15.0,0.0,0.0,0.0,6.0,0.0,3.0


In [5]:
data.insert(3, 'lanes', lanes_col)
data.insert(4, 'is_oneway', oneway_col)

In [6]:
data.columns

Index(['count_id', 'count_date', 'location_id', 'lanes', 'is_oneway',
       'location', 'lng', 'lat', 'centreline_type', 'centreline_id', 'px',
       'time_start', 'time_end', 'sb_cars_r', 'sb_cars_t', 'sb_cars_l',
       'nb_cars_r', 'nb_cars_t', 'nb_cars_l', 'wb_cars_r', 'wb_cars_t',
       'wb_cars_l', 'eb_cars_r', 'eb_cars_t', 'eb_cars_l', 'sb_truck_r',
       'sb_truck_t', 'sb_truck_l', 'nb_truck_r', 'nb_truck_t', 'nb_truck_l',
       'wb_truck_r', 'wb_truck_t', 'wb_truck_l', 'eb_truck_r', 'eb_truck_t',
       'eb_truck_l', 'sb_bus_r', 'sb_bus_t', 'sb_bus_l', 'nb_bus_r',
       'nb_bus_t', 'nb_bus_l', 'wb_bus_r', 'wb_bus_t', 'wb_bus_l', 'eb_bus_r',
       'eb_bus_t', 'eb_bus_l', 'nx_peds', 'sx_peds', 'ex_peds', 'wx_peds',
       'nx_bike', 'sx_bike', 'ex_bike', 'wx_bike', 'nx_other', 'sx_other',
       'ex_other', 'wx_other'],
      dtype='object')

In [7]:
row_dict = {
    'location_id': 0,
    'year': 0,
    'month': 0,
    'day': 0,
    'time_start_hour': 0,
    'time_start_min': 0,
    'time_end_hour': 0,
    'time_end_min': 0,
    'num_lanes': 0,
    'is_oneway': 0,
    'is_weekend': 0,
    'is_holiday': 0,
    #predict
    'nx': 0,
    'sx': 0,
    'ex': 0,
    'wx': 0,
    #predict
    'nb_r': 0,
    'nb_t': 0,
    'nb_l': 0,
    'sb_r': 0,
    'sb_t': 0,
    'sb_l': 0,
    'eb_r': 0,
    'eb_t': 0,
    'eb_l': 0,
    'wb_r': 0,
    'wb_t': 0,
    'wb_l': 0
}
data_list = []

def getTime(time):
    hour, minute, _ = time.split(' ')[1].split('-')[0].split(':')
    return float(hour), float(minute)

def isWeekend(date):
    return datetime.strptime(date, '%Y-%m-%d').weekday() > 4

def isHoliday(date):
    return datetime.strptime(date, '%Y-%m-%d') in holidays.CA()

for index, row in data.iterrows():
    row_dict['location_id'] = row['location_id']
    row_dict['year'], row_dict['month'], row_dict['day'] = row['count_date'].split('-')
    row_dict['time_start_hour'], row_dict['time_start_min'] = getTime(row['time_start'])
    row_dict['time_end_hour'], row_dict['time_end_min'] = getTime(row['time_end'])
    row_dict['num_lanes'] = row['lanes']
    row_dict['is_oneway'] = row['is_oneway']
    row_dict['is_weekend'] = isWeekend(row['count_date'])
    row_dict['is_holiday'] = isHoliday(row['count_date'])

    row_dict['nx'] = float(row['nx_peds']) + float(row['nx_bike']) + float(row['nx_other'])
    row_dict['sx'] = float(row['sx_peds']) + float(row['sx_bike']) + float(row['sx_other'])
    row_dict['ex'] = float(row['ex_peds']) + float(row['ex_bike']) + float(row['ex_other'])
    row_dict['wx'] = float(row['wx_peds']) + float(row['wx_bike']) + float(row['wx_other'])

    row_dict['nb_r'] = float(row['nb_cars_r']) + float(row['nb_truck_r']) + float(row['nb_bus_r'])
    row_dict['nb_t'] = float(row['nb_cars_t']) + float(row['nb_truck_t']) + float(row['nb_bus_t'])
    row_dict['nb_l'] = float(row['nb_cars_l']) + float(row['nb_truck_l']) + float(row['nb_bus_l'])
    
    row_dict['sb_r'] = float(row['sb_cars_r']) + float(row['sb_truck_r']) + float(row['sb_bus_r'])
    row_dict['sb_t'] = float(row['sb_cars_t']) + float(row['sb_truck_t']) + float(row['sb_bus_t'])
    row_dict['sb_l'] = float(row['sb_cars_l']) + float(row['sb_truck_l']) + float(row['sb_bus_l'])
    
    row_dict['eb_r'] = float(row['eb_cars_r']) + float(row['eb_truck_r']) + float(row['eb_bus_r'])
    row_dict['eb_t'] = float(row['eb_cars_t']) + float(row['eb_truck_t']) + float(row['eb_bus_t'])
    row_dict['eb_l'] = float(row['eb_cars_l']) + float(row['eb_truck_l']) + float(row['eb_bus_l'])
    
    row_dict['wb_r'] = float(row['wb_cars_r']) + float(row['wb_truck_r']) + float(row['wb_bus_r'])
    row_dict['wb_t'] = float(row['wb_cars_t']) + float(row['wb_truck_t']) + float(row['wb_bus_t'])
    row_dict['wb_l'] = float(row['wb_cars_l']) + float(row['wb_truck_l']) + float(row['wb_bus_l'])

    data_list.append(row_dict.copy())


In [8]:
finalData = pd.DataFrame(data_list)
finalData
finalData.to_csv('tmcs_2020_2029_final.csv', index=False)

In [4]:
# finalData
data_clean

,location_id,year,month,day,time_start_hour,time_start_min,time_end_hour,time_end_min,num_lanes,is_oneway,...,nb_l,sb_r,sb_t,sb_l,eb_r,eb_t,eb_l,wb_r,wb_t,wb_l
0,13060,2020,1,8,7.0,30.0,7.0,45.0,2,False,...,0.0,0.0,125.0,15.0,0.0,0.0,0.0,6.0,0.0,6.0
1,13060,2020,1,8,7.0,45.0,8.0,0.0,2,False,...,0.0,0.0,117.0,10.0,0.0,0.0,0.0,10.0,0.0,4.0
2,13060,2020,1,8,8.0,0.0,8.0,15.0,2,False,...,0.0,0.0,134.0,17.0,0.0,0.0,0.0,13.0,0.0,10.0
3,13060,2020,1,8,8.0,15.0,8.0,30.0,2,False,...,0.0,0.0,161.0,9.0,0.0,0.0,0.0,15.0,0.0,9.0
4,13060,2020,1,8,8.0,30.0,8.0,45.0,2,False,...,0.0,0.0,139.0,15.0,0.0,0.0,0.0,6.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27838,22573,2020,1,25,16.0,45.0,17.0,0.0,1,False,...,39.0,15.0,68.0,4.0,19.0,5.0,7.0,1.0,5.0,27.0
27839,22573,2020,1,25,17.0,0.0,17.0,15.0,1,False,...,26.0,17.0,38.0,4.0,14.0,10.0,17.0,2.0,12.0,25.0
27840,22573,2020,1,25,17.0,15.0,17.0,30.0,1,False,...,19.0,21.0,59.0,6.0,16.0,10.0,15.0,3.0,12.0,30.0
27841,22573,2020,1,25,17.0,30.0,17.0,45.0,1,False,...,20.0,11.0,59.0,3.0,11.0,8.0,7.0,2.0,15.0,37.0


In [4]:
def isWeekend(y, m, d):
    try:
        return datetime(y, m, d).weekday() > 4
    except ValueError:
        print(y, m, d)
        return False

def isHoliday(y, m, d):
    try:
        return datetime(y, m, d) in holidays.CA()
    except ValueError:
        print(y, m, d)
        return False

def prep_data_end_to_end(data, graph):
    row_dict = {
        'year': 0,
        'month': 0,
        'day': 0,
        'time_start_hour': 0,
        'time_start_min': 0,
        'time_end_hour': 0,
        'time_end_min': 0,
        'is_weekend': 0,
        'is_holiday': 0,
        'start': '',
        'end': ''
    }
    data_list = []
    starts_ends_list = []
    
    years = data['year'].unique()
    months = data['month'].unique()
    days = data['day'].unique()
    start_hours = data['time_start_hour'].unique()
    start_minutes = data['time_start_min'].unique()
    start_minutes = np.delete(start_minutes, np.argwhere(start_minutes == 1))
    end_hours = data['time_end_hour'].unique()
    end_minutes = data['time_end_min'].unique()
    end_minutes = np.delete(end_minutes, np.argwhere(end_minutes == 16))
    intersections = list(graph.columns)
    intersections.remove('Unnamed: 0')
    
    
    for start in intersections:
        for end in intersections:
            if end != start:
                starts_ends_list.append([start, end])
    
    for year in years:
        for month in months:
            for day in days:
                if (month == 2 and day > 29): continue
                is_weekend = isWeekend(year, month, day)
                is_holiday = isHoliday(year, month, day)
                for start_hour in start_hours:
                    for start_minute in start_minutes:
                        end_time = datetime.strptime(str(int(start_hour)) + ':' + str(int(start_minute)), '%H:%M') + dt.timedelta(minutes = 15)
                        for start, end in starts_ends_list:
                            row_dict['year'] = year
                            row_dict['month'] = month
                            row_dict['day'] = day
                            row_dict['time_start_hour'] = start_hour
                            row_dict['time_start_min'] = start_minute
                            row_dict['time_end_hour'] = end_time.hour
                            row_dict['time_end_min'] = end_time.minute
                            row_dict['is_weekend'] = is_weekend
                            row_dict['is_holiday'] = is_holiday
                            row_dict['start'] = start
                            row_dict['end'] = end

                            data_list.append(row_dict.copy())
    
                for end_hour in end_hours:
                    for end_minute in end_minutes:
                        start_time = datetime.strptime(str(int(end_hour)) + ':' + str(int(end_minute)), '%H:%M') + dt.timedelta(minutes = -15)
                        for start, end in starts_ends_list:
                            row_dict['year'] = year
                            row_dict['month'] = month
                            row_dict['day'] = day
                            row_dict['time_start_hour'] = start_time.hour
                            row_dict['time_start_min'] = start_time.minute
                            row_dict['time_end_hour'] = end_hour
                            row_dict['time_end_min'] = end_minute
                            row_dict['is_weekend'] = is_weekend
                            row_dict['is_holiday'] = is_holiday
                            row_dict['start'] = start
                            row_dict['end'] = end

                            data_list.append(row_dict.copy())
                                
    finalData = pd.DataFrame(data_list)
    finalData.to_csv('tmcs_2020_2029_end_to_end.csv', index = False)
        

In [8]:
a = [['13060', '16271'], ['13060', '12260'], ['13060', '5351'], ['13060', '4608'], ['13060', '24770'], ['13060', '41219'], ['13060', '41242']]
for i, j in a:
    print(i, j)

13060 16271
13060 12260
13060 5351
13060 4608
13060 24770
13060 41219
13060 41242


In [5]:
prep_data_end_to_end(data_clean, graph)

In [5]:
finalData

NameError: name 'finalData' is not defined

In [26]:
longest_path = 5
out_cols = pd.DataFrame(columns = list(range(1, longest_path + 1)))
paths = [
         [1,  2], 
         [12, 1,  14, 43, 3], 
         [1,  2,  3,  4], 
         [1,  1,  1], 
         [12, 54, 74, 23, 7]
        ]
data_clean.join(out_cols)
for i, path in enumerate(paths):
    for j in range(longest_path):
            try:
                data_clean.at[i, j + 1] = path[j]
            except:
                data_clean.at[i, j + 1] = 'pad'
data_clean

,location_id,year,month,day,time_start_hour,time_start_min,time_end_hour,time_end_min,num_lanes,is_oneway,...,eb_t,eb_l,wb_r,wb_t,wb_l,1,2,3,4,5
0,13060,2020,1,8,7.0,30.0,7.0,45.0,2,False,...,0.0,0.0,6.0,0.0,6.0,1.0,2.0,pad,pad,pad
1,13060,2020,1,8,7.0,45.0,8.0,0.0,2,False,...,0.0,0.0,10.0,0.0,4.0,12.0,1.0,14,43,3
2,13060,2020,1,8,8.0,0.0,8.0,15.0,2,False,...,0.0,0.0,13.0,0.0,10.0,1.0,2.0,3,4,pad
3,13060,2020,1,8,8.0,15.0,8.0,30.0,2,False,...,0.0,0.0,15.0,0.0,9.0,1.0,1.0,1,pad,pad
4,13060,2020,1,8,8.0,30.0,8.0,45.0,2,False,...,0.0,0.0,6.0,0.0,3.0,12.0,54.0,74,23,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27838,22573,2020,1,25,16.0,45.0,17.0,0.0,1,False,...,5.0,7.0,1.0,5.0,27.0,NaN,NaN,NaN,NaN,NaN
27839,22573,2020,1,25,17.0,0.0,17.0,15.0,1,False,...,10.0,17.0,2.0,12.0,25.0,NaN,NaN,NaN,NaN,NaN
27840,22573,2020,1,25,17.0,15.0,17.0,30.0,1,False,...,10.0,15.0,3.0,12.0,30.0,NaN,NaN,NaN,NaN,NaN
27841,22573,2020,1,25,17.0,30.0,17.0,45.0,1,False,...,8.0,7.0,2.0,15.0,37.0,NaN,NaN,NaN,NaN,NaN
